In [1]:
import os
import ee
import geemap
import math
Map = geemap.Map()
ee.Initialize()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [2]:
import urllib.request

In [3]:
#找到上传的数据，只需要保护区数据
geemap.ee_search()

In [4]:
def generateGrid(xmin, xmax, dx, y1, y2):
    xx = ee.List.sequence(xmin, ee.Number(xmax).subtract(dx), dx)
    cells = xx.map(func_uke).flatten()
    return ee.FeatureCollection(cells)

def func_uke(x):
    x1 = ee.Number(x)
    x2 = ee.Number(x).add(ee.Number(dx))
    coords = ee.List([x1, y1, x2, y2])
    rect = ee.Algorithms.GeometryConstructors.Rectangle(coords)
    return ee.Feature(rect)

In [5]:
def func_feh(feature):
    #自然植被                    
    arableland5 = feature.getNumber('arableland5')
    forest5 = feature.getNumber('forest5')
    grassland5 = feature.getNumber('grassland5')
    shrubland5 = feature.getNumber('shrubland5')
    wetlands5 = feature.getNumber('wetlands5')
    waterbody5 = feature.getNumber('waterbody5')
    tundra5 = feature.getNumber('tundra5')
    atificalsurface5 = feature.getNumber('atificalsurface5')
    bareground5 = feature.getNumber('bareground5')
    permanentsnow5 = feature.getNumber('permanentsnow5')
    Land5 = feature.getNumber('Land5')   
    #feature的面积
    return feature.set({
        #自然植被
        'arableland5':arableland5.divide(Land5),
        'forest5':forest5.divide(Land5),
        'grassland5':grassland5.divide(Land5),
        'shrubland5':shrubland5.divide(Land5),
        'wetlands5':wetlands5.divide(Land5),
        'waterbody5':waterbody5.divide(Land5),
        'tundra5':tundra5.divide(Land5),
        'atificalsurface5':atificalsurface5.divide(Land5),
        'bareground5':bareground5.divide(Land5),
        'permanentsnow5':permanentsnow5.divide(Land5),
        'Land5':Land5.divide(Land5)
     })

In [6]:
import requests
from retrying import retry
# 全部报错才会报错，如果其中一次正常，则继续执行
# 两次retry之间等待2秒，重试7次
@retry(stop_max_attempt_number=40, wait_fixed=2000)
def get_request(url):
    response = urllib.request.urlretrieve(url, filename=out_ndvi_stats)
    return response

In [7]:
dx = 1/120
xmin = 0
xmax = 180
ymin = -58
ymax = 90
yrange = round((ymax - ymin  - 0.0001)/dx)+1
print(yrange)

17761


In [8]:
lat_list = ['n_80_n','n_75_n','n_70_n','n_65_n','n_60_n','n_55_n','n_50_n','n_45_n','n_40_n','n_35_n','n_30_n','n_25_n','n_20_n','n_15_n','n_10_n','n_05_n','n_00_n','s_00_s','s_05_s','s_10_s','s_15_s','s_20_s','s_25_s','s_30_s','s_35_s','s_40_s','s_45_s','s_50_s','s_55_s']

In [9]:
#读取土地利用
for year_index in range(0,3):
    #range的Max改为18000
    for i in range (0,18000):
        try:          
            year = 2000 + 10*year_index
            lat_i = int(i/600)-1
            latitude = lat_list[lat_i]
            path = 'users/1801213219/Globeland30_2000/Globeland30'+latitude+'_'+str(year)
            image_lc = ee.Image(path)
            #合并土地利用
            arableland_pro= image_lc.gte(2)
            arableland = image_lc.updateMask(arableland_pro).lte(19).unmask(0)
            forest_pro= image_lc.gte(20)
            forest = image_lc.updateMask(forest_pro).lte(29).unmask(0)
            grassland_pro= image_lc.gte(30)
            grassland = image_lc.updateMask(grassland_pro).lte(39).unmask(0)
            shrubland_pro= image_lc.gte(40)
            shrubland = image_lc.updateMask(shrubland_pro).lte(49).unmask(0)
            wetlands_pro= image_lc.gte(50)
            wetlands = image_lc.updateMask(wetlands_pro).lte(59).unmask(0)
            waterbody_pro= image_lc.gte(60)
            waterbody = image_lc.updateMask(waterbody_pro).lte(69).unmask(0)
            tundra_pro= image_lc.gte(70)
            tundra = image_lc.updateMask(tundra_pro).lte(79).unmask(0)
            atificalsurface_pro= image_lc.gte(80)
            atificalsurface = image_lc.updateMask(atificalsurface_pro).lte(89).unmask(0)
            bareground_pro= image_lc.gte(90)
            bareground = image_lc.updateMask(bareground_pro).lte(99).unmask(0)
            permanentsnow_pro= image_lc.gte(100)
            permanentsnow = image_lc.updateMask(permanentsnow_pro).lte(120).unmask(0)
            Land_pro= image_lc.gte(2)
            Land = image_lc.updateMask(Land_pro).lte(120).unmask(0)
            cond = ee.Image.cat(
            arableland.eq(1).rename('arableland5'),
            forest.eq(1).rename('forest5'),
            grassland.eq(1).rename('grassland5'),
            shrubland.eq(1).rename('shrubland5'),
            wetlands.eq(1).rename('wetlands5'),
            waterbody.eq(1).rename('waterbody5'),
            tundra.eq(1).rename('tundra5'),
            atificalsurface.eq(1).rename('atificalsurface5'),
            bareground.eq(1).rename('bareground5'),
            permanentsnow.eq(1).rename('permanentsnow5'),
            Land.eq(1).rename('Land5'))
            i_g = yrange-i-2
            y1 = ymin + i_g*dx
            y2 = ymin + (i_g+1)*dx
            grid = generateGrid(xmin, xmax, dx, y1, y2)
            npsRes = cond.reduceRegions(grid, ee.Reducer.sum(), 60)#500不是分辨率，改动对结果影响小
            npsRes = npsRes.map(func_feh)
            name='Globeland30_'+'Globe_'+str(i)+'_Half2.csv'
            out_dir = 'G:\\LUCC_China\\Globeland30\\GEE_GlobeRatio\\Y'+str(year)+'\\Half2'
            out_ndvi_stats = os.path.join(out_dir, name)
            if not os.path.exists(out_dir):
                os.makedirs(out_dir)
            # Allowed output formats: csv, shp, json, kml, kmz
            # Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM, resolution=500
            #SCALE represent the resolution of the image that can be used in .multiply(ee.Image.pixelArea())
            url = npsRes.getDownloadURL(filetype="csv", selectors=['TARGET_FID','arableland5',
                                                                    'forest5',
                                                                    'grassland5',
                                                                    'shrubland5',
                                                                    'wetlands5',
                                                                    'waterbody5',
                                                                    'tundra5',
                                                                    'atificalsurface5',
                                                                    'bareground5',
                                                                    'permanentsnow5',
                                                                    'Land5'], filename=name)
            print(url)
            print(out_ndvi_stats)
            get_request(url)
        except Exception as e:
            print(e)       

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/9f53b700531935f0e38c8a175294fdd7-6f15076b459e7a36b812611fb47fe362:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_1980_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/a6eaebe81f0bb750d9c6aee16ee11be1-87f13bc8ddaa56eb1bf4171b6de3dcdb:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_1981_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/bbb8f39cfdf3e214c7dd894eb5f0a6b3-a1e6497d93af19c28334e1365d5d2da3:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_1982_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/9179119f2d4040c69d5f9d335b9d99ab-2f879411e5a97d684333ecd4eb709ebc:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_1983_Half2.csv
https://earthengine.googleapis.c

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Too Many Requests: Request was rejected because the request rate or concurrency limit was exceeded.
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/6bf7d5ea6646ff6902f22b04f485f17e-eb81ea7fcf02586c44e61c8618968533:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2024_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/37d39188ea714e0a83783fdcdd8c995a-2adcf7702c0bfcc433a36d2d3e85d481:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2025_Half2.csv
https://earthengine.googleapis.com/v1alpha/p

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/8f22677a44aab0059b77bb75a5286f2e-a689ed1499c20c51e11df5d938aece34:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2056_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/81bf2897bf740a23e68837e2cc51c26e-f03f8616e73ab6ae2370ed1494c9eca4:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2057_Half2.csv
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Too Many Requests: Request was rejected because the request rate or concurrency limit was exceeded.
Too Many Requests: Request was rejected because the request rate or concurrency limit was exceeded.
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/178f387e4077bf255b37b0e28fe70e08-fc9b398a05c2f3a1f69ac5fdd907e282:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Gl

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/5424fb7598a2afec40a86ead48720510-ab956ac85b1ce933fe0254fe4d384db9:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2093_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/da77c5373fab47cc8f551ef0bc194e50-df9e9641267bfdd29e8cb15493db9f77:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2094_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/1bc83c13d2963dc6d18e9e43484e9477-17ff583f57a1e9c96772c96793bfb277:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2095_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/56cfed5b007cfa0cb816523d8a092428-90f4d4b38c5835da6a80bc7644c964d2:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2096_Half2.csv
https://earthengine.googleapis.c

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/855a814f768f0bb3cfead8fc3de97c55-a839c25242341e8b1561a20c270eff2a:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2129_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/03c6ddb100d12f7f4f802b99ddcde96e-a5919c88ec240071a62456acfd2612f0:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2130_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/fab12f31195311b58258167015963b68-11acbba2f5fe99b40c2491b4f036814c:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2131_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/5e7b890f3c

...

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/db131f66144ba3abc03f49437eb9402d-71d29331a769239e643d472583dc6dbe:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2851_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/692487e37bc5931c83d44206e25ad2e6-53a0dcb5ebbb5e3606a9f6b658b1e38b:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2852_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/40e99ccda11a4f0a30c67ae45d2df074-e9ccc0711a38b907ccbf76724e23ae7f:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2853_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/01572cd1ca1cc477c48f2e4d8d2eb4ce-eb5ad71c98d60bfdd2eafb28cc4a67ea:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2854_Half2.csv
https://earthengine.googleapis.c

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/fd82a27389c6f559873a590e92e46aaf-b025823dfdaa6323fa9109447ceb86c3:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2885_Half2.csv
Computation timed out.
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/49cd3273481ac0ad781ad0e7d1b3a3c5-e92572e9971b448762bb54d1d49e58e1:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2887_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/c41a6f64a3e4b256cd108274424d9206-caf22ce4786e6ff4ef3f96b6e0a9ae8e:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2888_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/879f3546b4734df18fb4264dd90bcebd-be9dcd09d1801a2f1aee169bf6a58b37:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2889_Half2.csv
https://e

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/19dd7a83338ae96d752e7e441ffc6489-548842896dfdd3ae5da1a0cef4c4f012:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2923_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/5690bc470dc91fd1cacdfab0b91c9baf-7b4b8ec84d2338a1193e9cd65eec0692:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2924_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/18478650f967f34a5c0c84302bbd683f-dae30679727aacf0e9983ab80887274c:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2925_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/f08f90645f7e225e635cb5877b8b2ddc-43411d1739eed94dadf5416a851d7189:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2926_Half2.csv
https://earthengine.googleapis.c

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/4b94578408ae01f66c0685451a1cfc96-186cd4f5a955904eb8bfdae060506015:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2958_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/f3df60d8e9b6d8d139423c5ccd804ad1-30ff87f766343e925262f5423343b290:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2959_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/3e1aced72e98ba133bdbdbaa15c9911d-35f612476c1024de5453a191933c57fe:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2960_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/78066e2245a1151089c5c4663e7c0efc-96896243ceb79dcc06d350fa5ca6f491:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2961_Half2.csv
https://earthengine.googleapis.c

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/7b29fdefea6b342ba3020f6c9212d6b6-f08ed1272b4d93dac684dee7ac44d755:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2994_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/734921d7cb6e76adc5132becc9808daf-52dc2998c015a9a2a83d089e00fcda45:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2995_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/f3f54fb17ea81ac5ec5590e7dd36ab02-fbbfa16a0f77d7e43b1180980937295c:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2996_Half2.csv
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/cf7c330781b7a1054d520b82979787e1-72d12abc419f1d440c6615f21448758c:getFeatures
G:\LUCC_China\Globeland30\GEE_GlobeRatio\Y2000\Half2\Globeland30_Globe_2997_Half2.csv
https://earthengine.googleapis.c